# pyQuil - The Quantum Programing Toolkit in Python

This toolkit provides some simple libraries for writing quantum programs using the quantum instruction language Quil. pyQuil is part of the [Forest](http://forest.rigetti.com) suite of tools for quantum programming and is currently in beta.

```
import pyquil.quil as pq
import pyquil.forest as forest
from pyquil.gates import *
qvm = forest.Connection()
p = pq.Program()
p.inst(H(0), CNOT(0, 1))
    <pyquil.pyquil.Program object at 0x101ebfb50>
qvm.wavefunction(p)
    [(0.7071067811865475+0j), 0j, 0j, (0.7071067811865475+0j)]
```

It comes with a few parts:

1. **Quil**: The Quantum Instruction Language standard. Instructions written in Quil can be executed on any implementation of a quantum abstract machine, such as the quantum virtual machine (QVM), or on a real quantum processor unit (QPU). More details regarding Quil can be found in the [whitepaper](https://arxiv.org/abs/1608.03355).
2. **QVM**: A Quantum Virtual Machine: an implementation of the quantum abstract machine on classical hardware.  The QVM lets you use a regular computer to simulate a small quantum computer.  You can access the Rigetti QVM running in the cloud with your API key.  [Sign up here to get your key](http://forest.rigetti.com).
3. **pyQuil**: A Python library to help write and run Quil code and quantum programs in Python.

## Getting Started

### Prerequisites

Before starting, ensure that you have an installation of Python 2.7.(>= 10) and pip. We recommend installing [Anaconda](https://www.continuum.io/downloads) for an all-in-one installation. If pip is not installed, it can be installed with `easy_install pip`.

### Installation
After getting pyQuil from either a repository or from a source distribution, change to that directory in a terminal and run

```
pip install -e .
```

On Mac/Linux, if this command does not succeed because of permissions errors, then run with

```
sudo pip install -e .
```

The library will now be available globally.

### Connecting to a Rigetti QVM
In order to connect to a Rigetti QVM you need to configure your pyQuil installation with your QVM key. For permanent one time setup, you can do this by creating a file in your home directory with the following lines:
```
[Rigetti Forest]
url: <FOREST_URL>
key: <YOUR_FOREST_API_KEY>
```
Look [here](http://linktodocs.rigetti.com/connection) to see more information about setting up your connection to Forest.

If this configuration is not set, pyQuil will default to looking for a local QVM at `127.0.0.1:5000`.

For the temporarary purposes any notebook, you can set the configuration in enviornment variables.

In [10]:
!export QVM_URL=http://api.rigetti.com/qvm
!export QVM_API_KEY=JD1hxq1GEQFAfcFxKgx78HxrIAEmnp59QQs8Wdui

### Basic Usage

To ensure that your installation is working correctly, try running the following Python commands interactively.  First, import the `pyquil` module (which constructs quantum programs) and the `qvm` module (which allows connections to the Rigetti QVM). We'll also import some basic gates for pyQuil.


In [1]:
import pyquil.quil as pq
import pyquil.forest as forest
from pyquil.gates import *

Next, we want to open a connection to the QVM.

In [2]:
qvm = forest.Connection()

Now we can make a program by adding some Quil instruction using the `inst` method on a `Program` object.

In [3]:
p = pq.Program()
p.inst(X(0)).measure(0, 0)

This program simply applies the $X$-gate to the zeroth qubit, measures that qubit and stores the measurement result in the zeroth classical register. We can look at the Quil code that makes up this program simply by printing it.

In [4]:
print p

X 0
MEASURE 0 [0]



Most importantly, of course, we can see what happens if we run this program on the QVM with the following.

In [5]:
classical_regs = [0] # A list of which classical registers to return the values of.

qvm.run(p, classical_regs)

[[1]]

We see that the result of this program is that the classical register `[0]` now stores the state of qubit 0, which should be $\left\vert 1\right\rangle$ after an $X$-gate.  We can of course ask for more classical registers:

In [6]:
qvm.run(p, [0, 1, 2])

[[1, 0, 0]]

The classical registers are initialized to zero, so registers `[1]` and `[2]` come out as zero. If we stored the measurement in a different classical register we would obtain:

In [8]:
p = pq.Program()   # clear the old program
p.inst(X(0)).measure(0, 1)
qvm.run(p, [0, 1, 2])

[[0, 1, 0]]

We can also run programs multiple times and accumulate all the results in a single list. Try running this a few times to see different results.

In [9]:
coin_flip = pq.Program().inst(H(0)).measure(0, 0)
num_flips = 5
qvm.run(coin_flip, [0], num_flips)

[[0], [1], [0], [1], [0]]

Try running the above code several times. You will see that you will, with very high probability, get different results each time.

As the QVM is a virtual machine, we can also inspect the wavefunction of a program directly, even without measurements, e.g.

In [10]:
coin_flip = pq.Program().inst(H(0))
qvm.wavefunction(coin_flip)

array([ 0.70710678+0.j,  0.70710678+0.j])

It is important to remember that this is just a useful debugging tool for small quantum systems, and it cannot be feasibly obtained on a quantum processor.

### Some Program Construction Features

Multiple instructions can be applied at once or chained together. The following are all valid programs:

In [12]:
print "Multiple inst arguments with final measurement:"
print pq.Program().inst(X(0), Y(1), Z(0)).measure(0, 1)

print "Chained inst with explicit MEASURE instruction:"
print pq.Program().inst(X(0)).inst(Y(1)).measure(0, 1).inst(MEASURE(1, 2))

print "A mix of chained inst and measures:"
print pq.Program().inst(X(0)).measure(0, 1).inst(Y(1), X(0)).measure(0, 0)

print "A composition of two programs:"
print pq.Program(X(0)) + pq.Program(Y(0))

Multiple inst arguments with final measurement:
X 0
Y 1
Z 0
MEASURE 0 [1]

Chained inst with explicit MEASURE instruction:
X 0
Y 1
MEASURE 0 [1]
MEASURE 1 [2]

A mix of chained inst and measures:
X 0
MEASURE 0 [1]
Y 1
X 0
MEASURE 0 [0]

A composition of two programs:
X 0
Y 0



### Fixing a Mistaken Instruction

If an instruction was appended to a program incorrectly, one can pop it off.

In [13]:
p = pq.Program().inst(X(0))
p.inst(Y(1))
print "Oops! We have added Y 1 by accident:"
print p

print "We can fix by popping:"
p.pop()
print p

print "And then add it back:"
p += pq.Program(Y(1))
print p

Oops! We have added Y 1 by accident:
X 0
Y 1

We can fix by popping:
X 0

And then add it back:
X 0
Y 1



### Defining New Gates

New gates can be easily added inline to Quil programs. All you need is a matrix representation of the gate. For example, below we define a $\sqrt{X}$ gate.

In [14]:
import numpy as np
from scipy.linalg import sqrtm

# First we define the new gate from a matrix
x_gate_matrix = np.array(([0.0, 1.0], [1.0, 0.0]))
sqrt_x = sqrtm(x_gate_matrix)
p = pq.Program().defgate("SQRT-X", sqrt_x)

# Then we can use the new gate,
p.inst(("SQRT-X", 0))
print p

DEFGATE SQRT-X:
    0.49999999999999989+0.49999999999999989i, 0.49999999999999989-0.49999999999999989i
    0.49999999999999989-0.49999999999999989i, 0.49999999999999989+0.49999999999999989i

SQRT-X 0



In [15]:
qvm.wavefunction(p)

array([ 0.5+0.5j,  0.5-0.5j])

Quil in general supports defining parametric gates, though right now only static gates are supported by pyQuil. Below we show how we can define $X_1\otimes \sqrt{X_0} $ as a single gate.

In [16]:
# A multi-qubit defgate example
x_gate_matrix = np.array(([0.0, 1.0], [1.0, 0.0]))
x_sqrt_x = np.kron(sqrtm(x_gate_matrix), x_gate_matrix)
p = pq.Program().defgate("X-SQRT-X", x_sqrt_x)

# Then we can use the new gate
p.inst(("X-SQRT-X", 1, 0))
qvm.wavefunction(p)

array([ 0.0+0.j ,  0.5+0.5j,  0.0+0.j ,  0.5-0.5j])

### A More Advanced Example: QFT

Let's do an example that includes multi-qubit parameterized gates.

Here we wish to compute the discrete Fourier transform of `[0, 1, 0, 0, 0, 0, 0, 0]`. We do this in three steps:

1. Write a function called `qft3` to make a 3-qubit QFT quantum program.
2. Write a state preparation quantum program.
3. Execute state preparation followed by the QFT on the QVM.

First we define a function to make a 3-qubit QFT quantum program. This is a mix of Hadamard and CPhase gates, with a final bit reversal correction at the end consisting of a single Swap gate.

In [17]:
from math import pi

def qft3(q0, q1, q2):
    p = pq.Program()
    p.inst( H(q2),
            CPHASE(pi/2.0, q1, q2),
            H(q1),
            CPHASE(pi/4.0, q0, q2),
            CPHASE(pi/2.0, q0, q1),
            H(q0),
            SWAP(q0, q2) )
    return p

There is a very important detail to be recognized here: The function `qft3` doesn't *compute* the QFT, but rather it *makes a quantum program* to compute the QFT on qubits `q0`, `q1`, and `q2`.

We can look at what this program looks like in Quil notation by doing the following:

In [18]:
print qft3(0, 1, 2)

H 2
CPHASE(1.5707963267948966) 1 2
H 1
CPHASE(0.7853981633974483) 0 2
CPHASE(1.5707963267948966) 0 1
H 0
SWAP 0 2



Next, we want to prepare a state that corresponds to the sequence we want to compute the discrete Fourier transform of. Fortunately, this is easy, we just apply an X gate to the zero state.

In [19]:
state_prep = pq.Program().inst(X(0))

We can verify that this works by computing its wavefunction. However, we need to add some "dummy" qubits, because otherwise `wavefunction` would return a two-element vector.

In [20]:
add_dummy_qubits = pq.Program().inst(I(2))
qvm.wavefunction(state_prep + add_dummy_qubits)

array([ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j])

If we have two quantum programs `a` and `b`, we can concatenate them by doing `a + b`. Using this, all we need to do is compute the QFT after state preparation to get our final result.

In [21]:
qvm.wavefunction(state_prep + qft3(0, 1, 2))

array([  3.53553391e-01+0.j        ,   2.50000000e-01+0.25j      ,
         2.16489014e-17+0.35355339j,  -2.50000000e-01+0.25j      ,
        -3.53553391e-01+0.j        ,  -2.50000000e-01-0.25j      ,
        -2.16489014e-17-0.35355339j,   2.50000000e-01-0.25j      ])

We can verify this works by computing the (inverse) FFT from NumPy.

In [22]:
from numpy.fft import ifft
ifft([0,1,0,0,0,0,0,0], norm="ortho")

array([ 0.35355339+0.j        ,  0.25000000+0.25j      ,
        0.00000000+0.35355339j, -0.25000000+0.25j      ,
       -0.35355339+0.j        , -0.25000000-0.25j      ,
        0.00000000-0.35355339j,  0.25000000-0.25j      ])

### More Advanced Control Flow

Here are a couple quick examples that show how much richer the classical control of a Quil program can be. In this first example, we have a register called `classical_flag_register` which we use for looping. Then we construct the loop in the following steps:

1. We first initialize this register to `1` with the `init_register` program so our while loop will execute. This is often called the *loop preamble* or *loop initialization*.

2. Next, we write body of the loop in a program itself. This will be a program that computes an $X$ followed by an $H$ on our qubit.

3. Lastly, we put it all together using the `quil_while` method.

In [23]:
# Name our classical registers:
classical_flag_register = 2

# Write out the loop initialization and body programs:
init_register = pq.Program(X(0)).measure(0, classical_flag_register)
loop_body = pq.Program(X(0), H(0)).measure(0, classical_flag_register)

# Put it all together in a loop program:
loop_prog = init_register.while_do(classical_flag_register, loop_body)

print loop_prog

X 0
MEASURE 0 [2]
LABEL @START1
JUMP-WHEN @END2 [2]
X 0
H 0
MEASURE 0 [2]
JUMP @START1
LABEL @END2



In this next example, we show how to do conditional branching in the form of the traditional `if` construct as in many programming languages. Much like the last example, we construct programs for each branch of the `if`, and put it all together by using the `quil_if` method.

In [24]:
# Name our classical registers:
test_register = 1
answer_register = 0

# Construct each branch of our if-statement. We can have empty branches
# simply by having empty programs.
then_branch = pq.Program(X(0))
else_branch = pq.Program()

# Make a program that will put a 0 or 1 in test_register with 50% probability:
branching_prog = pq.Program(H(1)).measure(1, test_register)

# Add the conditional branching:
branching_prog.if_then(test_register, then_branch, else_branch)

# Measure qubit 0 into our answer register:
branching_prog.measure(0, answer_register)

print branching_prog

H 1
MEASURE 1 [1]
JUMP-WHEN @THEN3 [1]
JUMP @END4
LABEL @THEN3
X 0
LABEL @END4
MEASURE 0 [0]



We can run this program a few times to see what we get in the `answer_register`.

In [25]:
qvm.run(branching_prog, [answer_register], 10)

[[1], [1], [1], [0], [1], [0], [0], [1], [1], [0]]

## The Standard Gate Set
The following gates methods come standard with Quil and `gates.py`:

- Pauli gates  `I`, `X`, `Y`, `Z`

- Hadamard gate: `H`

- Phase gates: `PHASE(`$\theta$`)`, `S`, `T`

- Controlled phase gates: `CPHASE00(` $\alpha$ `)`, `CPHASE01(` $\alpha$ `)`, `CPHASE10(` $\alpha$ `)`, `CPHASE(` $\alpha$ `)`

- Cartesian rotation gates: `RX(` $\theta$ `)`, `RY(` $\theta$ `)`, `RZ(` $\theta$ `)`

- Controlled $X$ gates: `CNOT`, `CCNOT`

- Swap gates: `SWAP`, `CSWAP`, `ISWAP`, `PSWAP(` $\alpha$ `)`

The gates of a single argument take a real or complex floating point number as an argument.

### Advanced: Parametric Depolarizing Noise
The Rigetti QVM has support for emulating certain types of noise models. One such model is *parametric depolarizing noise*, which is defined by a set of 6 probabilities:

- The probabilities $P_X$, $P_Y$, and $P_Z$ which define respectively the probability of a Pauli $X$, $Y$, or $Z$ gate getting applied to *each* qubit after *every* gate application. These probabilities are called the *gate noise probabilities*.

- The probabilities $P_X'$, $P_Y'$, and $P_Z'$ which define respectively the probability of a Pauli $X$, $Y$, or $Z$ gate getting applied to the qubit being measured *before* it is measured. These probabilities are called the *measurement noise probabilities*.

We can instantiate a noisy QVM by creating a new connection with these probabilities specified.

In [26]:
# 20% chance of a X gate being applied after gate applications and before measurements.
gate_noise_probs = [0.2, 0.0, 0.0]
meas_noise_probs = [0.2, 0.0, 0.0]
noisy_qvm = qvm_endpoint.Connection(gate_noise=gate_noise_probs, measurement_noise=meas_noise_probs)

We can test this by applying an $X$ gate and measuring. Nominally, we should always measure `1`.

In [27]:
p = pq.Program().inst(X(0)).measure(0, 0)
print "Without Noise:", qvm.run(p, [0], 10)
print "With Noise   :", noisy_qvm.run(p, [0], 10)

Without Noise: [[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]
With Noise   : [[0], [0], [0], [0], [0], [1], [1], [1], [1], [0]]


### Advanced: Pauli Operator Algebra
Many algorithms require manipulating sums of Pauli combinations, such as \\[\sigma = \tfrac{1}{2}I - \tfrac{3}{4}X_0Y_1Z_3 + (5-2i)Z_1X_2,\\] where $G_n$ indicates the gate $G$ acting on qubit $n$. We can represent such sums by constructing `PauliTerm` and `PauliSum`. The above sum can be constructed as follows:

In [29]:
from pyquil.paulis import ID, sX, sY, sZ

# Pauli term takes an operator "X", "Y", "Z", or "I"; a qubit to act on, and
# an optional coefficient.
a = 0.5 * ID
b = -0.75 * sX(0) * sY(1) * sZ(3)
c = (5-2j) * sZ(1) * sX(2)

# Construct a sum of Pauli terms.
sigma = a + b + c
print "sigma =", sigma

sigma = 0.5*I + -0.75*X0*Y1*Z3 + (5-2j)*Z1*X2


There are two primary things one can do with Pauli terms and sums:

1. A Pauli sum's fully "tensored up" form can be computed with the `tensor_up` function.

2. Quil code can be generated to compute the exponentiation of a Pauli term, i.e., $\exp[-i\sigma]$.

When arithmetic is done with Pauli sums, simplification is automatically done.

The following shows an instructive example of all three.

In [38]:
import pyquil.paulis as pl

# Simplification
sigma_cubed = sigma * sigma * sigma
print "Simplified  :", sigma_cubed
print

#Produce Quil code to compute exp[iX]
H = -1.0 * sX(0)
print "Quil to compute exp[iX] on qubit 0:"
print pl.exponential_map(H)(1.0)

Simplified  : (32.46875-30j)*I + (-16.734375+15j)*X0*Y1*Z3 + (71.5625-144.625j)*Z1*X2

Quil to compute exp[iX] on qubit 0:
H 0
RZ(-2.0) 0
H 0



## Exercises
### Exercise 1.

Write a quantum program to simulate throwing an 8-sided die.  The Python function you should produce is:

```
def throw_octahedral_die():
    # return the result of throwing an 8 sided die, an int between 1 and 8, by running a quantum program
```

Next, extend the program to work for any kind of fair die:

```
def throw_polyhedral_die(num_sides):
    # return the result of throwing a num_sides sided die by running a quantum program
```

### Exercise 2.

We can use the full generality of NumPy and SciPy to construct new gate matrices.

1. Write a function `controlled` which takes a $2\times 2$ matrix $U$ representing a single qubit operator, and makes a $4\times 4$ matrix which is a controlled variant of $U$, with the first argument being the *control qubit*.

2. Write a Quil program to define a controlled-$Y$ gate in this manner. Find the wavefunction when applying this gate to qubit 1 controlled by qubit 0.

### Exercise 3.

Write a quantum program for the single-shot Grover's algorithm.  The Python function you should produce is:

```
# data is an array of 0's and 1's such that there are exactly three times as many
# 0's as 1's
def single_shot_grovers(data):
    # return an index that contains the value 1
```

As an example: `single_shot_grovers([0,0,1,0])` should return 2.

#### Hints
Remember that the Grover's diffusion operator is:

$$
\begin{pmatrix}
2/N - 1 & 2/N & \cdots & 2/N \\
2/N &  & &\\
\vdots & & \ddots & \\
2/N & & & 2/N-1
\end{pmatrix}
$$